In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 21.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=70e930f751bf8072a6a69e58ab875e4dbe8469cd18fcde01393c80c033844c5e
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 24.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [4]:
import pandas as pd
df = pd.read_csv('5차년도_2차.csv',encoding='cp949')
df

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,fear,happiness,1,sadness,1,sadness,2,sadness,1,sadness,1,23,female
19370,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,angry,sadness,1,fear,1,sadness,2,sadness,1,neutral,0,23,female
19371,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,fear,sadness,1,neutral,0,sadness,2,fear,1,sadness,1,23,female
19372,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,happiness,disgust,1,sadness,1,neutral,0,happiness,1,sadness,1,23,female


In [5]:
np.unique(df.상황)

array(['angry', 'disgust', 'fear', 'happiness', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [6]:
condition = (df.상황 == 'angry') | (df.상황 == 'disgust') | (df.상황 == 'fear') | (df.상황 == 'sadness')

In [8]:
df2 = df[condition]
df2

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
10,5f3ca01b8a3c1005aa97c4d3,나 요즘 너무 우울해 죽겠어.,sadness,sadness,2,sadness,2,sadness,2,sadness,1,sadness,2,48,female
11,5f3ca06b8a3c1005aa97c4da,맞아. 사람은 나가서 활동을 해야 된다고.,sadness,neutral,0,sadness,1,sadness,2,sadness,1,sadness,2,48,female
12,5f3ca09d8a3c1005aa97c4df,좀 활기 있는 영화로 좀 부탁할게.,sadness,neutral,0,sadness,1,sadness,1,sadness,1,sadness,1,48,female
13,5f3ca4778a3c1005aa97c50d,내가 좋아하는 유튜버가 좋은 제품이라고 하길래 나도 샀지. 그런데 그게 돈 받고 하...,angry,disgust,2,angry,2,angry,1,angry,1,angry,1,48,female
14,5f3ca4a28a3c1005aa97c510,그래서 사람들이 증거 사진을 올렸더니 그 때서야 광고였다고 미안하다고 하는 거야.,angry,disgust,2,angry,2,angry,1,angry,1,angry,1,48,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,5fbe25444c55eb78bd7cedb6,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,angry,sadness,1,sadness,1,neutral,0,sadness,1,neutral,0,23,female
19369,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,fear,happiness,1,sadness,1,sadness,2,sadness,1,sadness,1,23,female
19370,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,angry,sadness,1,fear,1,sadness,2,sadness,1,neutral,0,23,female
19371,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,fear,sadness,1,neutral,0,sadness,2,fear,1,sadness,1,23,female


In [9]:
df2.loc[(df2['상황'] == "angry"), '상황'] = 0  
df2.loc[(df2['상황'] == "disgust"), '상황'] = 0  
df2.loc[(df2['상황'] == "fear"), '상황'] = 1  
df2.loc[(df2['상황'] == "sadness"), '상황'] = 2  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
np.unique(df2.상황)

array([0, 1, 2], dtype=object)

In [11]:
9818 * 0.8

7854.400000000001

In [12]:
train_df = df2.iloc[:7860, 1:3]
train_df

,발화문,상황
10,나 요즘 너무 우울해 죽겠어.,2
11,맞아. 사람은 나가서 활동을 해야 된다고.,2
12,좀 활기 있는 영화로 좀 부탁할게.,2
13,내가 좋아하는 유튜버가 좋은 제품이라고 하길래 나도 샀지. 그런데 그게 돈 받고 하...,0
14,그래서 사람들이 증거 사진을 올렸더니 그 때서야 광고였다고 미안하다고 하는 거야.,0
...,...,...
15463,"곰팡이 제거제도 써봤는데, 별로야. 잘 안 없어져.",0
15464,와. 이놈의 곰팡이 어떡하지? 좋은 방법 없어?,0
15465,"아니, 문을 열어도 곰팡이가 계속 생겨.",0
15466,"아니, 이놈의 화장실은. 청소를 해도 곰팡이가 사라지질 않아.",0


In [13]:
test_df = df2.iloc[7860:, 1:3]
test_df

,발화문,상황
15468,전화해서 뭐라고 말해야되지?,0
15469,"와, 진짜 안 그래도 지금 버리려고.",0
15470,"아, 그러니까. 진짜 어떡하지?",0
15471,"어. 와, 진짜 자기는 하는 것도 없으면서 나한테만 그래.",0
15473,"아니, 그래도 짜증나는 건 짜증나는 거잖아. 그 새끼 진짜 별로야.",0
...,...,...
19368,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,0
19369,나 엘리베이터에 갇혔어.,1
19370,하지만 기분이 나쁜 걸 어떡해?,0
19371,자취방 엘리베이턴데 정전인가봐.,1


In [14]:
train = []
for q, label in zip(train_df['발화문'], train_df['상황'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train.append(data)

In [15]:
test = []
for q, label in zip(test_df['발화문'], test_df['상황'])  :
    data = []
    data.append(q)
    data.append(str(label))

    test.append(data)

In [16]:
train[0]

['나 요즘 너무 우울해 죽겠어.', '2']

In [17]:
test[0]

['전화해서 뭐라고 말해야되지?', '0']

In [18]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [20]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      

In [23]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [24]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [25]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [26]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [28]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/123 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.0971029996871948 train acc 0.359375
epoch 1 train acc 0.7365736397748593


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/31 [00:00<?, ?it/s]

...

epoch 9 test acc 0.9164898132427843


  0%|          | 0/123 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0007436372106894851 train acc 1.0
epoch 10 train acc 0.995045731707317


  0%|          | 0/31 [00:00<?, ?it/s]

epoch 10 test acc 0.9159857809847198


In [ ]:
# 테스트 문장 예측
test_sentence = '그 상사는 정말 화가 나네요!!'
test_label = 0

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['대화 내용', '감정']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 5.9649, -2.8059, -2.4276]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


참고 코드
https://github.com/aeddung/ML-DL/blob/main/NLP/koBert_multiclassification.py

참고 사이트 

*   https://moondol-ai.tistory.com/241 
*   https://doheon.github.io/%EC%BD%94%EB%93%9C%EA%B5%AC%ED%98%84/nlp/ci-kobert-post/

